In [ ]:
!pip install botorch
!pip3 install pyro-ppl
!pip install scipy==1.4.1

import numpy as np 
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.linalg as spla
import numpy.linalg as npla
import sklearn.gaussian_process as gp
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.test_functions import Hartmann
from botorch.utils.transforms import unnormalize
import math
import gpytorch
from matplotlib import pyplot as plt
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal
from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition import ProbabilityOfImprovement
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.acquisition.analytic import ExpectedImprovement
# Dictionary for parameters
ker_vars = [10**(-4), 10**(-3), 10**(-2), 10**(-1), 1, 10**(1), 10**(2), 10**(3), 10**(4), 10**(5), 10**(6)]
ker_vars = np.array(ker_vars)
n_learners = 11
params = {'tau': 0.2*np.ones(n_learners) ,
          'nu': 50*np.ones(n_learners), 'kern_shutdown_flg': 10**(-16),
          'ker_params': ker_vars, 'M': n_learners, 'n_RF': 50,
          'prob_bias': 0, 'sigma_bias': 0}


In [ ]:
!pip install Box2D
!pip install pygame
from push_world import *
import sys
import random

# Copyright (c) 2017 Zi Wang
def robot_pushing_3d(
    rx: float,
    ry: float,
    duration: float,
    ox: float,
    oy: float,
) -> float:
    simu_steps = int(10 * duration)
    # set it to False if no gui needed
    world = b2WorldInterface(False)
    oshape, osize, ofriction, odensity, bfriction, hand_shape, hand_size  = 'circle', 1, 0.01, 0.05, 0.01, 'rectangle', (0.3,1) 
    #thing,base = make_thing(500, 500, world, oshape, osize, ofriction, odensity, bfriction, (ox,oy))
    thing,base = make_thing(500, 500, world, oshape, osize, ofriction, odensity, bfriction, (0,0))

    init_angle = np.arctan(ry/rx)
    robot = end_effector(world, (rx,ry), base, init_angle, hand_shape, hand_size)
    final_location = simu_push(world, thing, robot, base, simu_steps)
    ret = np.linalg.norm(np.array([ox, oy]) - final_location)
    return ret

def robot_pushing_4d(
    rx: float,
    ry: float,
    duration: float,
    init_angle: float,
    ox: float,
    oy: float,
) -> float:
    simu_steps = int(10 * duration)
    # set it to False if no gui needed
    world = b2WorldInterface(False)
    oshape, osize, ofriction, odensity, bfriction, hand_shape, hand_size  = 'circle', 1, 0.01, 0.05, 0.01, 'rectangle', (0.3,1) 
    #thing,base = make_thing(500, 500, world, oshape, osize, ofriction, odensity, bfriction, (ox,oy))
    thing,base = make_thing(500, 500, world, oshape, osize, ofriction, odensity, bfriction, (0,0))
    xvel = -rx;
    yvel = -ry;
    regu = np.linalg.norm([xvel,yvel])
    xvel = xvel / regu * 10;
    yvel = yvel / regu * 10;
    init_angle = np.arctan(ry/rx)
    robot = end_effector(world, (rx,ry), base, init_angle, hand_shape, hand_size)
    final_location = simu_push2(world, thing, robot, base, xvel, yvel, simu_steps)
    ret = np.linalg.norm(np.array([ox, oy]) - final_location)
    return ret
    
def robot_pushing_2x2(
    rx: float,
    ry: float,
    xvel: float,
    yvel: float,
    duration: float,
    init_angle: float,
    rx2: float,
    ry2 : float,
    xvel2: float,
    yvel2: float,
    duration2: float,
    init_angle2: float,
    rtor: float,
    rtor2: float,
    gx: float,
    gy: float,
    gx2: float,
    gy2: float,
    
) -> float:
    simu_steps = int(10 * duration)
    simu_steps2 = int(10 * duration2)
    # set it to False if no gui needed    
    world = b2WorldInterface(False)
    oshape, osize, ofriction, odensity, bfriction, hand_shape, hand_size  = 'circle', 1, 0.01, 0.05, 0.01, 'rectangle', (1,0.3) #'circle', 0.3#
    #thing,base = make_thing(500, 500, world, oshape, osize, ofriction, odensity, bfriction, (0,0))
    base = make_base(500, 500, world)
    thing = make_1thing(base, world, 'rectangle', (0.5,0.5), ofriction, odensity, (0, 2))
    thing2 = make_1thing(base, world, 'circle', 1, ofriction, odensity, (0,-2))
    #xvel = np.cos(init_angle)*5;
    #yvel = np.sin(init_angle)*5;
    robot = end_effector(world, (rx,ry), base, init_angle, hand_shape, hand_size)
    robot2 = end_effector(world, (rx2,ry2), base, init_angle2, hand_shape, hand_size)
    (ret1, ret2) = simu_push_2robot2thing(world, thing, thing2, robot, robot2, base, xvel, yvel, xvel2, yvel2, rtor, rtor2, simu_steps, simu_steps2)
    #print ret1, ret2
    ret1 = np.linalg.norm(np.array([gx, gy]) - ret1)
    ret2 = np.linalg.norm(np.array([gx2, gy2]) - ret2)
    return ret1+ret2

In [ ]:
# Create dataset 

def syn_fun(X, syn_function, tau):

  if syn_function == "robot_push3d":
    x1 = X[:,0]
    x2 = X[:,1]
    x3 = X[:,2]
    des_pos = [-4.143838793976, -3.59614858230749]
    y = []
    for i in range(X.shape[0]):
      y.append(-robot_pushing_3d(x1[i],x2[i],x3[i],des_pos[0],des_pos[1])) 
    y += random.normalvariate(0, tau)
    return y
  elif syn_function == "robot_push4d":
    x1 = X[:,0]
    x2 = X[:,1]
    x3 = X[:,2]
    x4 = X[:,3]
    des_pos = [-2.406383184994124, 4.831160212812405]
    y = []
    for i in range(X.shape[0]):
      y.append(-robot_pushing_4d(x1[i],x2[i],x3[i],x4[i],des_pos[0],des_pos[1])) 
    y += random.normalvariate(0, tau)
    return y


def obj_syn_fun(X, syn_function):

  if syn_function == "robot_push3d":
    x1 = X[:,0]
    x2 = X[:,1]
    x3 = X[:,2]
    des_pos = [-4.143838793976, -3.59614858230749]
    y = []
    for i in range(X.shape[0]):
      y.append(-robot_pushing_3d(x1[i],x2[i],x3[i],des_pos[0],des_pos[1]))  
    return y
  elif syn_function == "robot_push4d":
    x1 = X[:,0]
    x2 = X[:,1]
    x3 = X[:,2]
    x4 = X[:,3]
    des_pos =  [-2.406383184994124, 4.831160212812405]
    y = []
    for i in range(X.shape[0]):
      y.append(-robot_pushing_4d(x1[i],x2[i],x3[i],x4[i],des_pos[0],des_pos[1])) 
    return y


syn_function = "robot_push3d"
reinit_key = 0

if syn_function == "robot_push3d":
  bnds = ((-5,5), (-5,5), (0.99,30))
  benchmark_value = 0
  X = np.zeros((10,3))               #each row -> feature vector 
  X0_points = np.zeros((40,3))
  for i in range(40):
    if i < 10:
      X[i,0] = random.uniform(-5, 5)
      X[i,1] = random.uniform(-5, 5)
      X[i,2] = random.uniform(1, 30)

    X0_points[i,0] = random.uniform(-5,5)
    X0_points[i,1] = random.uniform(-5,5)
    X0_points[i,2] = random.uniform(1,30)
  low_bound = [-5, -5, 1]
  up_bound = [5, 5,30]

elif syn_function == "robot_push4d":
  bnds = ((-5,5), (-5,5), (0.99,30),(0.001,2*np.pi))
  benchmark_value = 0
  X = np.zeros((10,4))              #each row -> feature vector 
  X0_points = np.zeros((40,4))
  for i in range(40):
    if i < 10:
      X[i,0] = random.uniform(-5, 5)
      X[i,1] = random.uniform(-5, 5)
      X[i,2] = random.uniform(1, 30)
      X[i,3] = random.uniform(0, 2*np.pi)
    X0_points[i,0] = random.uniform(-5,5)
    X0_points[i,1] = random.uniform(-5,5)
    X0_points[i,2] = random.uniform(1,30)
    X0_points[i,3] = random.uniform(0,2*np.pi)
  low_bound = [-5, -5, 1,0]
  up_bound = [5, 5,30, 2*np.pi]


print("===============================================================")
print('X=',X)
tau = params["tau"][0]
y = syn_fun(X, syn_function, tau)
print('y=',y)
print("===============================================================")


In [4]:
import math
import torch
import gpytorch
import pyro
from pyro.infer.mcmc import NUTS, MCMC, HMC
from matplotlib import pyplot as plt





Full Bayesian GP for hyperparams

In [5]:
from gpytorch.priors import LogNormalPrior, NormalPrior, UniformPrior
def full_Bayesian_hyperparams(X,params,num_samples,warmup_steps,num_points):
  train_X = torch.from_numpy(X)
  tau = params["tau"][0]
  y = syn_fun(X, syn_function, tau)
  train_Y = torch.from_numpy(y)
  train_Y = torch.reshape(train_Y,(num_points,1))
  # We will use the simplest form of GP model, exact inference
  class ExactGPModel(gpytorch.models.ExactGP):
      def __init__(self, train_x, train_y, likelihood):
          super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
          self.mean_module = gpytorch.means.ConstantMean()
          self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

      def forward(self, x):
          mean_x = self.mean_module(x)
          covar_x = self.covar_module(x)
          return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
  # Use a positive constraint instead of usual GreaterThan(1e-4) so that LogNormal has support over full range.
  likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())
  model = ExactGPModel(train_X, train_Y, likelihood)
  model.mean_module.register_prior("mean_prior", UniformPrior(0, 0.0001), "constant")
  model.covar_module.base_kernel.register_prior("lengthscale_prior", UniformPrior(10**(-3), 10**3), "lengthscale")
  model.covar_module.register_prior("outputscale_prior", UniformPrior(1, 100), "outputscale")
  likelihood.register_prior("noise_prior", UniformPrior(0.15, 0.25), "noise")

  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

  def pyro_model(x, y):
      with gpytorch.settings.fast_computations(False, False, False):
          sampled_model = model.pyro_sample_from_prior()
          output = sampled_model.likelihood(sampled_model(x))
          pyro.sample("obs", output, obs=y)
      return y

  nuts_kernel = NUTS(pyro_model)
  #mcmc_run = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps, disable_progbar=smoke_test)
  mcmc_run = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
  mcmc_run.run(train_X, train_Y)
  #model.pyro_load_from_samples(mcmc_run.get_samples())
  samples = mcmc_run.get_samples()
  ker_var_vec = samples["covar_module.base_kernel.lengthscale_prior"].numpy()
  ker_var_vec = np.reshape(ker_var_vec,(1,num_samples))
  nu_vec = samples["covar_module.outputscale_prior"].numpy()
  nu_vec = np.reshape(nu_vec,(1,num_samples))
  tau_vec = samples["likelihood.noise_prior"].numpy()
  tau_vec = np.reshape(tau_vec,(1,num_samples))
  return ker_var_vec,nu_vec,tau_vec




Codes for RF-based single GP & EGPs

In [ ]:
def GP_hyp_param_optim(X,y,ker_var,ker_lscale_type,tau,syn_function):
  
  if ker_lscale_type == "fixed":
    kernel = 1.0 * gp.kernels.RBF(length_scale=1.0, length_scale_bounds=(ker_var, ker_var))
    model_gp = gp.GaussianProcessRegressor(kernel=kernel,alpha=tau,n_restarts_optimizer=10,normalize_y=False)
    model_gp.fit(X, y)
    learned_kernel = model_gp.kernel_
    temp = learned_kernel.get_params(deep=True)["k1"]
    nu = temp.get_params()["constant_value"]
    ker_len = ker_var 
  else:
    kernel = 1.0 * gp.kernels.RBF(length_scale=1.0)
    model_gp = gp.GaussianProcessRegressor(kernel=kernel,alpha=tau,n_restarts_optimizer=10,normalize_y=False)
    model_gp.fit(X, y)
    learned_kernel = model_gp.kernel_
    ker_len = learned_kernel.get_params()["k2__length_scale"]
    temp = learned_kernel.get_params(deep=True)["k1"]
    nu = temp.get_params()["constant_value"]
  return nu, ker_len

def RF_gen(len,D,ker_var):
  V = np.random.randn(D,len)
  V = (1/np.sqrt(ker_var))*V
  return V 

def RF_phi(x,V,D):
  Vtx = V@x  
  phi_xt=(1/np.sqrt(D))*np.vstack((np.cos(Vtx),np.sin(Vtx)))
  phi_xt = np.reshape(phi_xt,(2*D,1))
  return phi_xt

def single_predict(phi_xt1, Sigma_t, tau, theta_t):
  mu_p = np.dot(theta_t.T, phi_xt1)
  var_p = np.dot(phi_xt1.T, Sigma_t@phi_xt1) + tau
  return mu_p, var_p

def single_correct(theta,Sigma,sig_hat_sq,phi_xt1,y_t1,y_hat_t1,D,sigma_bias):
  sig_h_inv = (1/sig_hat_sq)
  Sphi = Sigma@phi_xt1
  G = sig_h_inv*(Sphi)
  theta = theta+G*(y_t1-y_hat_t1)
  Sigma_new = Sigma-sig_hat_sq*(G@G.T)
  return theta, Sigma_new

def gauss_sample(theta_t,Sigma):
  mean = np.squeeze(theta_t)
  cov = Sigma
  samp = np.random.multivariate_normal(mean,cov, 1)
  return samp.T


def optimize_x(theta_samp,V,D,X0_points,bnds):
  func = lambda x: -np.dot(RF_phi(x,V,D).T,theta_samp)
  n_points = X0_points.shape[0]
  best_x = []
  best_fun = -np.inf 
  for i in range(n_points):
    x0 = X0_points[i,:]
    res = minimize(func, x0, method="L-BFGS-B", bounds=bnds)
    if -res.fun > best_fun:
      best_x = res.x
      best_fun = -res.fun
  return best_x, best_fun

    
def simple_regret(obj_f_list, benchmark_value):
  regret_list = []
  for i in range(len(obj_f_list)):
    regret_list.append(np.max(obj_f_list[0:i+1]))
  regret_list = benchmark_value*np.ones(len(obj_f_list)) - regret_list
  return regret_list 

def post_update_init(X,y,V,D,sigma_bias,tau,theta_t,Sigma):
  n_points = X.shape[0]
  len_x = X.shape[1]
  for i in range(n_points):
    x_t = X[i,:]
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    y_t = y[i]
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
  return theta_t, Sigma 

      

def main_GP(X,y,params,X0_points,bnds,syn_function, reinit_key):
  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  sigma_bias = params["sigma_bias"]
  tau = tau_vec[0]
  ker_vars = params["ker_params"]
  ker_var = ker_vars[3]              #Just for initialization
  [nu, ker_var] = GP_hyp_param_optim(X,y,ker_var,"nfixed",tau,syn_function)
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  # Initialization of theta_t and Sigma of the posterior 
  theta_t = np.zeros((2*D,1))
  Sigma = nu*np.identity(2*D)
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  # Generate vectors v for the RF
  V = RF_gen(len_x,D,ker_var)         
  theta_t, Sigma = post_update_init(X,y,V,D,sigma_bias,tau,theta_t,Sigma)
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)         
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    
    if reinit_key == 1:
      if t%1 == 0:
        [nu, ker_var] = GP_hyp_param_optim(X_list,y_list,ker_var,"nfixed",tau,syn_function)
        theta_t = np.zeros((2*D,1))
        Sigma = nu*np.identity(2*D)
        V = RF_gen(len_x,D,ker_var)
        theta_t, Sigma = post_update_init(X_list,y_list,V,D,sigma_bias,tau,theta_t,Sigma)

    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    y_pred.append(mu_p)
    var_p = np.squeeze(var_p)
    var_pred.append(var_p)
    # Correction step
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    # Sample from posterior 
    theta_samp = gauss_sample(theta_t,Sigma)
    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}
    [x_t, obj_x_t] = optimize_x(theta_samp,V,D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))

  return x_t,y_pred,obj_f_list,var_pred

def myGL(y,mu,var):
  gl = (1/(np.sqrt(2*np.pi*var)))*np.exp(-((y-mu)**2)/(2*var))
  return gl

def update_weights(w_vec_in,l_vec,prob_bias):
  w_vec_out=w_vec_in * l_vec
  if (np.sum(w_vec_out)) == 0:
    w_vec_out = w_vec_in
  else:
    w_vec_out = w_vec_out/(np.sum(w_vec_out))
    w_vec_out = w_vec_out+prob_bias
    w_vec_out = w_vec_out/(np.sum(w_vec_out))
  return w_vec_out 

def main_EGP(X,y,params,X0_points,bnds,syn_function, reinit_key):
  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]             #for noisy function measurements
  ker_vars = params["ker_params"]
  M = params["M"]
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]

  # Initialization of theta_t of posterior for all M experts
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  for m in range(M):
    # Hyperparameter tuning for all M learners
    [nu_vec[m], ker_vars[m]] = GP_hyp_param_optim(X,y,ker_vars[m],"fixed",tau_vec[m],syn_function)

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  w_vec=(1/M)*np.ones((1,M))         #Initialization of weights 
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  l_vec = np.zeros((1,M))
  mu_vec = np.zeros((1,M))
  var_vec = np.zeros((1,M))
  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    V[m,:,:] = RF_gen(len_x,D,ker_vars[m])       
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    
    if reinit_key == 1:
      if t%50 == 0:
        theta_t = np.zeros((M,2*D,1))
        for m in range(M):
          [nu_vec[m], ker_vars[m]] = GP_hyp_param_optim(X_list,y_list,ker_vars[m],"fixed",tau_vec[m],syn_function)
          Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
          V[m,:,:] = RF_gen(len_x,D,ker_vars[m])
          theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X_list,y_list,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
          
    for m in range(M):
      # Construct RF vector for x_t 
      phi_xt = RF_phi(x_t,V[m,:,:],D)
      Sigma[m,:,:] = Sigma[m,:,:] + sigma_bias*np.identity(2*D)
      # Prediction step
      [mu_p, var_p]=single_predict(phi_xt, Sigma[m,:,:], tau_vec[m], theta_t[m,:,:])
      mu_p = np.squeeze(mu_p)
      mu_vec[0,m] = mu_p
      var_p = np.squeeze(var_p)
      var_vec[0,m] = var_p
      # Correction step
      [theta_t[m,:,:],Sigma[m,:,:]]=single_correct(theta_t[m,:,:],Sigma[m,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)
      gl = myGL(y_t,mu_p,var_p)
      gl = np.squeeze(gl)
      l_vec[0,m] = myGL(y_t,mu_p,var_p)
  
    w_vec=update_weights(w_vec,l_vec,prob_bias)
    # Choose specific expert based on w pmf 
    w_pmf = np.squeeze(w_vec)
    ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)
 
    # Sample from posterior 
    theta_samp = gauss_sample(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:])

    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}
 
    [x_t, obj_x_t] = optimize_x(theta_samp,V[ind_exp-1,:,:],D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))

  return x_t,y_pred,obj_f_list,var_pred

 
  

EGPs with different types of kernels combined

In [7]:

# include Matern kernel

def GP_hyp_param_optim_matern(X,y,ker_var,ker_lscale_type,tau,syn_function,nu_mat):
  
  if ker_lscale_type == "fixed":
    kernel = 1.0 * gp.kernels.Matern(length_scale=1.0, length_scale_bounds=(ker_var, ker_var), nu = nu_mat)
    model_gp = gp.GaussianProcessRegressor(kernel=kernel,alpha=tau,n_restarts_optimizer=10,normalize_y=False)
    model_gp.fit(X, y)
    learned_kernel = model_gp.kernel_
    temp = learned_kernel.get_params(deep=True)["k1"]
    nu2 = temp.get_params()["constant_value"]
    ker_len = ker_var 
  else:
    kernel = 1.0 * gp.kernels.Matern(length_scale=1.0, nu = nu_mat)
    model_gp = gp.GaussianProcessRegressor(kernel=kernel,alpha=tau,n_restarts_optimizer=10,normalize_y=False)
    model_gp.fit(X, y)
    learned_kernel = model_gp.kernel_
    ker_len = learned_kernel.get_params()["k2__length_scale"]
    temp = learned_kernel.get_params(deep=True)["k1"]
    nu2 = temp.get_params()["constant_value"]
  return nu2, ker_len

# include ARD kernel 

def GP_hyp_param_optim_ARD(X,y,len_x,tau,syn_function):
  kernel = 1.0 * gp.kernels.RBF(length_scale= np.ones(len_x))
  model_gp = gp.GaussianProcessRegressor(kernel=kernel,alpha=tau,n_restarts_optimizer=10,normalize_y=False)
  model_gp.fit(X, y)
  learned_kernel = model_gp.kernel_
  ker_len = learned_kernel.get_params()["k2__length_scale"]
  temp = learned_kernel.get_params(deep=True)["k1"]
  nu = temp.get_params()["constant_value"]
  return nu, ker_len

def RF_gen_ARD(len,D,ker_var_vec):
  V = np.random.randn(D,len)
  V = V/np.sqrt(ker_var_vec)
  return V 

def RF_gen_Matern(len,D,ker_var,nu_mat):
  V = np.random.randn(D,len)
  V = (1/np.sqrt(ker_var))*V
  V = V/np.sqrt(np.random.gamma(shape=nu_mat, scale=1.0 / nu_mat, size=(D, 1)))
  return V 


def main_GP_matern(X,y,params,X0_points,bnds,syn_function, reinit_key):
  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]
  ker_vars = params["ker_params"]
  ker_var = ker_vars[3]
  [nu, ker_var] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau,syn_function,nu_mat=2.5)
  sigma_bias = params["sigma_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  # Initialization of theta_t and Sigma of the posterior 
  theta_t = np.zeros((2*D,1))
  Sigma = nu*np.identity(2*D)
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  # Generate vectors v for the RF
  V = RF_gen_Matern(len_x,D,ker_var,nu_mat = 2.5)        
  theta_t, Sigma = post_update_init(X,y,V,D,sigma_bias,tau,theta_t,Sigma)
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    
    if reinit_key == 1:
      if t%1 == 0:
        [nu, ker_var] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau,syn_function,nu_mat=2.5)
        theta_t = np.zeros((2*D,1))
        Sigma = nu*np.identity(2*D)
        V = RF_gen_Matern(len_x,D,ker_var,nu_mat = 2.5)
        theta_t, Sigma = post_update_init(X_list,y_list,V,D,sigma_bias,tau,theta_t,Sigma)

    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    y_pred.append(mu_p)
    var_p = np.squeeze(var_p)
    var_pred.append(var_p)
    # Correction step
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    # Sample from posterior 
    theta_samp = gauss_sample(theta_t,Sigma)

    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}

    [x_t, obj_x_t] = optimize_x(theta_samp,V,D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))

  return x_t,y_pred,obj_f_list,var_pred

def main_GP_matern2(X,y,params,X0_points,bnds,syn_function, reinit_key):

  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]
  ker_vars = params["ker_params"]
  ker_var = ker_vars[3]
  [nu, ker_var] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau,syn_function,nu_mat=1.5)
  sigma_bias = params["sigma_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  # Initialization of theta_t and Sigma of the posterior 
  theta_t = np.zeros((2*D,1))
  Sigma = nu*np.identity(2*D)
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  # Generate vectors v for the RF
  V = RF_gen_Matern(len_x,D,ker_var,nu_mat = 1.5)        
  theta_t, Sigma = post_update_init(X,y,V,D,sigma_bias,tau,theta_t,Sigma)
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    
    if reinit_key == 1:
      if t%1 == 0:
        [nu, ker_var] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau,syn_function,nu_mat=1.5)
        theta_t = np.zeros((2*D,1))
        Sigma = nu*np.identity(2*D)
        V = RF_gen_Matern(len_x,D,ker_var,nu_mat = 2.5)
        theta_t, Sigma = post_update_init(X_list,y_list,V,D,sigma_bias,tau,theta_t,Sigma)

    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    #print("y_pred=", mu_p)
    y_pred.append(mu_p)
    var_p = np.squeeze(var_p)
    var_pred.append(var_p)
    # Correction step
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    # Sample from posterior 
    theta_samp = gauss_sample(theta_t,Sigma)
    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}

    [x_t, obj_x_t] = optimize_x(theta_samp,V,D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))

  return x_t,y_pred,obj_f_list,var_pred

def main_GP_ARD(X,y,params,X0_points,bnds,syn_function, reinit_key):

  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]
  [nu, ker_vars] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau,syn_function)
  sigma_bias = params["sigma_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  # Initialization of theta_t and Sigma of the posterior 
  theta_t = np.zeros((2*D,1))
  Sigma = nu*np.identity(2*D)
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  # Generate vectors v for the RF
  V = RF_gen_ARD(len_x,D,ker_vars)
  theta_t, Sigma = post_update_init(X,y,V,D,sigma_bias,tau,theta_t,Sigma)
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    
    if reinit_key == 1:
      if t%1 == 0:
        [nu, ker_vars] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau,syn_function)
        theta_t = np.zeros((2*D,1))
        Sigma = nu*np.identity(2*D)
        V = RF_gen_ARD(len_x,D,ker_vars)
        theta_t, Sigma = post_update_init(X_list,y_list,V,D,sigma_bias,tau,theta_t,Sigma)


    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    y_pred.append(mu_p)
    var_p = np.squeeze(var_p)
    var_pred.append(var_p)
    # Correction step
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    # Sample from posterior 
    theta_samp = gauss_sample(theta_t,Sigma)
 
    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}

    [x_t, obj_x_t] = optimize_x(theta_samp,V,D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))

  return x_t,y_pred,obj_f_list,var_pred

def main_EGP_rbfmaternard(X,y,params,X0_points,bnds,syn_function, reinit_key):
  
  M = 4 
  nu_vec = np.ones(M)
  D = params["n_RF"]
  tau_vec = 0.2*np.ones(M)
  ker_var = 0.1 #initialization
  ker_vars = [] 
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  # Ensemble of 4 distinct kernels: 1) ARD 2) RBF 3) Matern v=1.5 4) Matern v=2.5
  for m in range(M):
    # Hyperparameter tuning for all M learners
    if m==0: 
      [nu_vec[m], temp] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau_vec[m],syn_function)
      temp = np.squeeze(temp)
      ker_vars.append(temp)
    elif m==1:
      [nu_vec[m], temp] = GP_hyp_param_optim(X,y,ker_var,"nfixed",tau_vec[m],syn_function)
      ker_vars.append(np.squeeze(temp))
    elif m==2:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=1.5)
      ker_vars.append(np.squeeze(temp))
    else:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=2.5)
      ker_vars.append(np.squeeze(temp))

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  w_vec=(1/M)*np.ones((1,M))         #Initialization of weights 
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = x_t.T
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  l_vec = np.zeros((1,M))
  mu_vec = np.zeros((1,M))
  var_vec = np.zeros((1,M))
  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    if m==0:
      V[m,:,:] = RF_gen_ARD(len_x,D,ker_vars[m])
    elif m==1:
      V[m,:,:] = RF_gen(len_x,D,ker_vars[m])
    elif m==2:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 1.5)
    else:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 2.5)

    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)

    if reinit_key == 1:
      if t%50 == 0:
        theta_t = np.zeros((M,2*D,1))
        Sigma = np.zeros((M,2*D,2*D))
        # Ensemble of 4 distinct kernels: 1) ARD 2) RBF 3) Matern v=1.5 4) Matern v=2.5
        for m in range(M):
          # Hyperparameter tuning for all M learners
          if m==0: 
            [nu_vec[m], temp] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau_vec[m],syn_function)
            temp = np.squeeze(temp)
            ker_vars.append(temp)
          elif m==1:
            [nu_vec[m], temp] = GP_hyp_param_optim(X,y,ker_var,"nfixed",tau_vec[m],syn_function)
            ker_vars.append(np.squeeze(temp))
          elif m==2:
            [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=1.5)
            ker_vars.append(np.squeeze(temp))
          else:
            [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=2.5)
            ker_vars.append(np.squeeze(temp))

          # Initialization of Sigma of posterior for all M experts
          Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)

    for m in range(M):
      # Construct RF vector for x_t 
      phi_xt = RF_phi(x_t,V[m,:,:],D)
      Sigma[m,:,:] = Sigma[m,:,:] + sigma_bias*np.identity(2*D)
      # Prediction step
      [mu_p, var_p]=single_predict(phi_xt, Sigma[m,:,:], tau_vec[m], theta_t[m,:,:])
      mu_p = np.squeeze(mu_p)
      mu_vec[0,m] = mu_p
      var_p = np.squeeze(var_p)
      var_vec[0,m] = var_p
      # Correction step
      [theta_t[m,:,:],Sigma[m,:,:]]=single_correct(theta_t[m,:,:],Sigma[m,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)
      gl = myGL(y_t,mu_p,var_p)
      gl = np.squeeze(gl)
      l_vec[0,m] = myGL(y_t,mu_p,var_p)
    
    w_vec=update_weights(w_vec,l_vec,prob_bias)
    # Choose specific expert based on w pmf 
    w_pmf = np.squeeze(w_vec)
    ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)

    # Sample from posterior 
    theta_samp = gauss_sample(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:])
 
    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}

    [x_t, obj_x_t] = optimize_x(theta_samp,V[ind_exp-1,:,:],D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))
    
  return x_t,y_pred,obj_f_list,var_pred


Code for RF-based Bandit BO


In [8]:
def split_data(X,y,M): 
  [T,len_x] = X.shape
  T_m = int(T/M)
  X_init = np.reshape(X,(M,T_m,len_x))
  y_init = np.reshape(y,(M,T_m,1))
  return X_init,y_init

def main_BanditBO(X,y,params,X0_points,bnds,syn_function,reinit_key):

  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]
  ker_vars = params["ker_params"]
  M = params["M"]

  # Initialization of theta_t of posterior for all M experts
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))

  for m in range(M):
    # Hyperparameter tuning for all M learners
    [nu_vec[m], ker_vars[m]] = GP_hyp_param_optim(X,y,ker_vars[m],"fixed",tau_vec[m],syn_function)

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  y_t = syn_fun(x_t.T, syn_function, tau)

  y_pred = []
  var_pred = []
  obj_f_list = []

  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    V[m,:,:] = RF_gen(len_x,D,ker_vars[m])       
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):  
    fcstar_list = []
    x_cand_list = np.zeros((M,len_x))
    for m in range(M):
      # Draw sample from posterior for bandit arm m
      theta_samp = gauss_sample(theta_t[m,:,:],Sigma[m,:,:])
      # Obtain x_tilde_m for bandit arm m
      [x_t_m, ts_val] = optimize_x(theta_samp,V[m,:,:],D,X0_points,bnds)
      ts_val = np.squeeze(ts_val)
      fcstar_list.append(ts_val)
      x_cand_list[m,:] = x_t_m
    
    # Choose arm that maximizes fcstar_list
    m_star = np.argmax(fcstar_list)

    x_t = x_cand_list[m_star,:]
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    y_t = syn_fun(x_t.T, syn_function, tau)

    # Update posterior params. of the chosen bandit arm
    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V[m_star,:,:],D)
    Sigma[m_star,:,:] = Sigma[m_star,:,:] + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma[m_star,:,:], tau_vec[m_star], theta_t[m_star,:,:])
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    # Correction step
    [theta_t[m_star,:,:],Sigma[m_star,:,:]]=single_correct(theta_t[m_star,:,:],Sigma[m_star,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)

  return x_t,y_pred,obj_f_list,var_pred


def main_BanditBO_ardrbfmatern(X,y,params,X0_points,bnds,syn_function,reinit_key):

  M = 4 
  nu_vec = np.ones(M)
  D = params["n_RF"]
  tau_vec = 0.2*np.ones(M)
  ker_var = 0.1 #initialization
  ker_vars = [] 
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  # Ensemble of 4 distinct kernels: 1) ARD 2) RBF 3) Matern v=1.5 4) Matern v=2.5
  for m in range(M):
    # Hyperparameter tuning for all M learners
    if m==0: 
      [nu_vec[m], temp] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau_vec[m],syn_function)
      temp = np.squeeze(temp)
      ker_vars.append(temp)
    elif m==1:
      [nu_vec[m], temp] = GP_hyp_param_optim(X,y,ker_var,"nfixed",tau_vec[m],syn_function)
      ker_vars.append(np.squeeze(temp))
    elif m==2:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=1.5)
      ker_vars.append(np.squeeze(temp))
    else:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=2.5)
      ker_vars.append(np.squeeze(temp))

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  y_t = syn_fun(x_t.T, syn_function, tau)

  y_pred = []
  var_pred = []
  obj_f_list = []
  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    if m==0:
      V[m,:,:] = RF_gen_ARD(len_x,D,ker_vars[m])
    elif m==1:
      V[m,:,:] = RF_gen(len_x,D,ker_vars[m])
    elif m==2:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 1.5)
    else:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 2.5)
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):
    fcstar_list = []
    x_cand_list = np.zeros((M,len_x))
    for m in range(M):
      # Draw sample from posterior for bandit arm m
      theta_samp = gauss_sample(theta_t[m,:,:],Sigma[m,:,:])
      # Obtain x_tilde_m for bandit arm m
      [x_t_m, ts_val] = optimize_x(theta_samp,V[m,:,:],D,X0_points,bnds)
      ts_val = np.squeeze(ts_val)
      fcstar_list.append(ts_val)
      x_cand_list[m,:] = x_t_m
    
    # Choose arm that maximizes fcstar_list
    m_star = np.argmax(fcstar_list)

    x_t = x_cand_list[m_star,:]
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    y_t = syn_fun(x_t.T, syn_function, tau)

    # Update posterior params. of the chosen bandit arm
    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V[m_star,:,:],D)
    Sigma[m_star,:,:] = Sigma[m_star,:,:] + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma[m_star,:,:], tau_vec[m_star], theta_t[m_star,:,:])
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    # Correction step
    [theta_t[m_star,:,:],Sigma[m_star,:,:]]=single_correct(theta_t[m_star,:,:],Sigma[m_star,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)

  return x_t,y_pred,obj_f_list,var_pred

Code for EXP3BO

In [9]:
import math
def get_reward(theta_t_m, X_list_m, w, V, D, T, len_x): 
  max = np.NINF
  for t in range(T):
    if math.isnan(X_list_m[t,0]) != True:
      x = np.array(X_list_m[t,:])
      x = np.reshape(x,(len_x,1))
      phi_x_t_m = RF_phi(x,V,D)
      cand = np.dot(phi_x_t_m.T,theta_t_m)
      if cand > max:
        max = cand
  reward = max/w
  return reward
  
def update_omega(omega,reward,eta,M):
  omega = omega*np.exp(eta*reward/M)
  return omega


def main_EXP3BO(X,y,params,X0_points,bnds,syn_function,reinit_key, benchmark_value):

  nu_vec = params["nu"]
  D = params["n_RF"]
  tau_vec = params["tau"]
  tau = tau_vec[0]
  ker_vars = params["ker_params"]
  M = params["M"]

  # Initialization of theta_t of posterior for all M experts
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  for m in range(M):

    # Hyperparameter tuning for all M learners
    [nu_vec[m], ker_vars[m]] = GP_hyp_param_optim(X,y,ker_vars[m],"fixed",tau_vec[m],syn_function)

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  y_t = syn_fun(x_t.T, syn_function, tau)
  omega_vec = np.ones((1,M))              #initialize omegas
  y_pred = []
  var_pred = []
  obj_f_list = []
  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    V[m,:,:] = RF_gen(len_x,D,ker_vars[m])       
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  eta = np.sqrt(M*np.log(M)/((np.e-1)*iter))
  X_list = np.empty((M,iter+1,len_x))
  X_list[:] = np.NaN
  for m in range(M):
    X_list[m,0,:] = x_t.T

  for t in range(iter):

    # Compute weights/ probabilities of experts 
    w_vec = (1-eta)*omega_vec/np.sum(omega_vec) + eta/M
    w_pmf = np.squeeze(w_vec)
  
    # Draw expert from weight pmf
    ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)

    # Sample from posterior 
    theta_samp = gauss_sample(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:])
    # obtain x 
    [x_t, obj_x_t] = optimize_x(theta_samp,V[ind_exp-1,:,:],D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    y_t = syn_fun(x_t.T, syn_function, tau)
    # include x_t to X_list for chosen expert
    X_list[ind_exp-1,t+1,:] = x_t.T

    # Update posterior params. of the chosen expert
    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V[ind_exp-1,:,:],D)
    Sigma[ind_exp-1,:,:] = Sigma[ind_exp-1,:,:] + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma[ind_exp-1,:,:], tau_vec[ind_exp-1], theta_t[ind_exp-1,:,:])
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    # Correction step
    [theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:]]=single_correct(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    #get reward
    reward = get_reward(theta_t[ind_exp-1,:,:], X_list[ind_exp-1,:,:], w_vec[0,ind_exp-1], V[ind_exp-1,:,:], D, iter+1, len_x)
    if benchmark_value!= 0:
      reward = reward/benchmark_value #normalize
    # update omegas
    omega_vec[0,ind_exp-1] = update_omega(omega_vec[0,ind_exp-1],reward,eta,M)
    
  return x_t,y_pred,obj_f_list,var_pred


def main_EXP3BO_ardrbfmatern(X,y,params,X0_points,bnds,syn_function,reinit_key, benchmark_value):

  M = 4 
  nu_vec = np.ones(M)
  D = params["n_RF"]
  tau_vec = 0.2*np.ones(M)
  ker_var = 0.1 #initialization
  ker_vars = [] 
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  # Ensemble of 4 distinct kernels: 1) ARD 2) RBF 3) Matern v=1.5 4) Matern v=2.5
  for m in range(M):
    # Hyperparameter tuning for all M learners
    if m==0: 
      [nu_vec[m], temp] = GP_hyp_param_optim_ARD(X,y,X.shape[1],tau_vec[m],syn_function)
      temp = np.squeeze(temp)
      ker_vars.append(temp)
    elif m==1:
      [nu_vec[m], temp] = GP_hyp_param_optim(X,y,ker_var,"nfixed",tau_vec[m],syn_function)
      ker_vars.append(np.squeeze(temp))
    elif m==2:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=1.5)
      ker_vars.append(np.squeeze(temp))
    else:
      [nu_vec[m], temp] = GP_hyp_param_optim_matern(X,y,ker_var,"nfixed",tau_vec[m],syn_function,nu_mat=2.5)
      ker_vars.append(np.squeeze(temp))

    # Initialization of Sigma of posterior for all M experts
    Sigma[m,:,:] = nu_vec[m]*np.identity(2*D)
  
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  y_t = syn_fun(x_t.T, syn_function, tau)
  omega_vec = np.ones((1,M))              #initialize omegas
  y_pred = []
  var_pred = []
  obj_f_list = []
  V = np.zeros((M,D,len_x))
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    if m==0:
      V[m,:,:] = RF_gen_ARD(len_x,D,ker_vars[m])
    elif m==1:
      V[m,:,:] = RF_gen(len_x,D,ker_vars[m])
    elif m==2:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 1.5)
    else:
      V[m,:,:] = RF_gen_Matern(len_x,D,ker_vars[m],nu_mat = 2.5)      
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_init(X,y,V[m,:,:],D,sigma_bias,tau_vec[m],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  eta = np.sqrt(M*np.log(M)/((np.e-1)*iter))
  X_list = np.empty((M,iter+1,len_x))
  X_list[:] = np.NaN
  for m in range(M):
    X_list[m,0,:] = x_t.T

  for t in range(iter):

    # Compute weights/ probabilities of experts 
    w_vec = (1-eta)*omega_vec/np.sum(omega_vec) + eta/M
    w_pmf = np.squeeze(w_vec)
  
    # Draw expert from weight pmf
    ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)

    # Sample from posterior 
    theta_samp = gauss_sample(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:])
    # obtain x 
    [x_t, obj_x_t] = optimize_x(theta_samp,V[ind_exp-1,:,:],D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)
    y_t = syn_fun(x_t.T, syn_function, tau)
    # include x_t to X_list for chosen expert
    X_list[ind_exp-1,t+1,:] = x_t.T

    # Update posterior params. of the chosen expert
    # Construct RF vector for x_t 
    phi_xt = RF_phi(x_t,V[ind_exp-1,:,:],D)
    Sigma[ind_exp-1,:,:] = Sigma[ind_exp-1,:,:] + sigma_bias*np.identity(2*D)
    # Prediction step
    [mu_p, var_p]=single_predict(phi_xt, Sigma[ind_exp-1,:,:], tau_vec[ind_exp-1], theta_t[ind_exp-1,:,:])
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    # Correction step
    [theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:]]=single_correct(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:],var_p,phi_xt,y_t,mu_p,D,sigma_bias)
    #get reward
    reward = get_reward(theta_t[ind_exp-1,:,:], X_list[ind_exp-1,:,:], w_vec[0,ind_exp-1], V[ind_exp-1,:,:], D, iter+1, len_x)
    if benchmark_value!= 0:
      reward = reward/benchmark_value #normalize

    # update omegas
    omega_vec[0,ind_exp-1] = update_omega(omega_vec[0,ind_exp-1],reward,eta,M)
    
    


  return x_t,y_pred,obj_f_list,var_pred

Code for Full Bayesian GP


In [10]:
def post_update_2(X,V,D,sigma_bias,tau,theta_t,Sigma):
  n_points = X.shape[0]
  len_x = X.shape[1]
  y = syn_fun(X, syn_function, tau)
  for i in range(n_points):
    x_t = X[i,:]
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    phi_xt = RF_phi(x_t,V,D)
    Sigma = Sigma + sigma_bias*np.identity(2*D)
    [mu_p, var_p]=single_predict(phi_xt, Sigma, tau, theta_t)
    mu_p = np.squeeze(mu_p)
    var_p = np.squeeze(var_p)
    y_t = y[i]
    [theta_t,Sigma]=single_correct(theta_t,Sigma,var_p,phi_xt,y_t,mu_p,D,sigma_bias)
  return theta_t, Sigma 

def main_full_Bayesian_GP(X,y,params,X0_points,bnds,syn_function):
  nu_init = params["nu"]
  D = params["n_RF"]
  tau_init = params["tau"]
  M = params["M"]
  sigma_bias = params["sigma_bias"]
  prob_bias = params["prob_bias"]
  kern_shutdown_flg=params["kern_shutdown_flg"]
  w_vec=(1/M)*np.ones((1,M))         #Initialization of weights (will remain unaltered)
  x_t = X[0,:]
  x_t = np.array(x_t)
  len_x = x_t.shape[0]
  x_t = np.reshape(x_t,(len_x,1)) 
  X_list = X
  y_pred = []
  var_pred = []
  y_list = []
  obj_f_list = []
  l_vec = np.zeros((1,M))
  mu_vec = np.zeros((1,M))
  var_vec = np.zeros((1,M))
  V = np.zeros((M,D,len_x))
  # Find hyperparams in a Bayesian way using all initial data
  ker_vars,nu_vec,tau_vec = full_Bayesian_hyperparams(X,params,num_samples=11,warmup_steps=80,num_points= X.shape[0])
  # Initialization of theta_t of posterior for all M experts
  theta_t = np.zeros((M,2*D,1))
  Sigma = np.zeros((M,2*D,2*D))
  for m in range(M):
    Sigma[m,:,:] = nu_vec[0,m]*np.identity(2*D)
  for m in range(M):
    # Generate vectors v for the RF for all M experts
    V[m,:,:] = RF_gen(len_x,D,ker_vars[0,m])       
    # Update theta and Sigma of posterior for all M experts using all init. data
    theta_t[m,:,:], Sigma[m,:,:] = post_update_2(X,V[m,:,:],D,sigma_bias,tau_init[0],theta_t[m,:,:],Sigma[m,:,:])
  iter = 200                           # iter: number of iterations/function evaluations 
  for t in range(iter):

    y_t = syn_fun(x_t.T, syn_function, tau)
    y_list.append(y_t)
    obj_val = obj_syn_fun(x_t.T, syn_function)
    obj_f_list.append(obj_val)

    # Choose specific expert based on w pmf 
    w_pmf = np.squeeze(w_vec)
    ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)

    # Sample from posterior 
    theta_samp = gauss_sample(theta_t[ind_exp-1,:,:],Sigma[ind_exp-1,:,:])
 
    # Acquisition function theta_samp * RF  (Thomson sampling)
    # Optimize acquisition function to obtain x_{t+1}

    [x_t, obj_x_t] = optimize_x(theta_samp,V[ind_exp-1,:,:],D,X0_points,bnds)
    x_t = np.array(x_t)
    x_t = np.reshape(x_t,(len_x,1))
    X_list = np.vstack((X_list, x_t.T))
    # Estimate new kernels hyperparams using all data gathered so far
    ker_vars,nu_vec,tau_vec = full_Bayesian_hyperparams(X_list,params,num_samples=11,warmup_steps=80,num_points= X_list.shape[0])
    # Initialization of theta_t of posterior for all M experts
    theta_t = np.zeros((M,2*D,1))
    Sigma = np.zeros((M,2*D,2*D))
    for m in range(M):
      Sigma[m,:,:] = nu_vec[0,m]*np.identity(2*D)
    V = np.zeros((M,D,len_x))
    for m in range(M):
      # Generate vectors v for the RF for all M experts
      V[m,:,:] = RF_gen(len_x,D,ker_vars[0,m])       
      # Update theta and Sigma of posterior for all M experts using all gathered data so far
      theta_t[m,:,:], Sigma[m,:,:] = post_update_2(X_list,V[m,:,:],D,sigma_bias,tau_init[0],theta_t[m,:,:],Sigma[m,:,:])

  return x_t,y_pred,obj_f_list,var_pred

In [11]:
import warnings
warnings.filterwarnings("ignore")

def update_weights_EI(log_w):
  log_w_aux = -np.sort(-log_w)
  log_w_aux_temp = log_w_aux[0]+np.log(1+np.sum(np.exp(log_w_aux[1:]-log_w_aux[0])))
  w = np.exp(log_w-log_w_aux_temp)
  return w


def BO_step_EI(train_X,train_Y, low_bound, up_bound, syn_function, tau):
  
  train_Y2 = torch.squeeze(train_Y)

  # Botorch GP
  gp = SingleTaskGP(train_X, train_Y)
  mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
  fit_gpytorch_model(mll)

  # acquistion functions

  EI = ExpectedImprovement(gp, best_f= torch.max(train_Y))

  #Optimize acquisition functions
  
  bounds = torch.stack([torch.zeros(train_X.shape[1])+torch.tensor(low_bound), torch.zeros(train_X.shape[1])+torch.tensor(up_bound)])
  candidate_EI, acq_value_EI = optimize_acqf(
    EI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
  )
  candidate_EI_num = candidate_EI.numpy()
  candidate_EI_num = np.float64(candidate_EI_num)
  score_EI = obj_syn_fun(candidate_EI_num, syn_function)
  obs_EI = syn_fun(candidate_EI_num, syn_function, tau)
  score_EI = np.array(score_EI)
  obs_EI = np.array(obs_EI)
  score_EI = torch.from_numpy(score_EI)
  obs_EI = torch.from_numpy(obs_EI)


  return candidate_EI, score_EI, obs_EI


def BO_EI(train_X,train_Y, low_bound, up_bound, syn_function):
  tau_vec = params["tau"]
  tau = tau_vec[0]
  temp = train_X.numpy() 
  F_eval = obj_syn_fun(temp, syn_function)
  F_eval = np.array(F_eval)
  F_eval = torch.from_numpy(F_eval)
  F_eval = F_eval[0:2]
  iter = 200
  for j in range(iter):
    [Xstar, sc, obs] = BO_step_EI(train_X,train_Y,low_bound,up_bound, syn_function, tau)
    train_X = torch.cat((train_X,Xstar),0)
    obs = torch.reshape(obs ,(1,1))
    train_Y = torch.cat((train_Y,obs),0)
    F_eval = torch.cat((F_eval, sc))
  obj_f_list = F_eval.numpy()
  return obj_f_list

def BO_step_EI_RBF(train_X,train_Y, low_bound, up_bound, syn_function, tau):
  
  train_Y2 = torch.squeeze(train_Y)

  # Botorch GP
  cov_module = gpytorch.kernels.RBFKernel()
  gp = SingleTaskGP(train_X, train_Y, covar_module = cov_module)
  mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
  fit_gpytorch_model(mll)
  

  # acquistion functions

  EI = ExpectedImprovement(gp, best_f= torch.max(train_Y))

  #Optimize acquisition functions
  
  bounds = torch.stack([torch.zeros(train_X.shape[1])+torch.tensor(low_bound), torch.zeros(train_X.shape[1])+torch.tensor(up_bound)])
  candidate_EI, acq_value_EI = optimize_acqf(
    EI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
  )
  candidate_EI_num = candidate_EI.numpy()
  candidate_EI_num = np.float64(candidate_EI_num)
  score_EI = obj_syn_fun(candidate_EI_num, syn_function)
  obs_EI = syn_fun(candidate_EI_num, syn_function, tau)
  score_EI = np.array(score_EI)
  obs_EI = np.array(obs_EI)
  score_EI = torch.from_numpy(score_EI)
  obs_EI = torch.from_numpy(obs_EI)


  return candidate_EI, score_EI, obs_EI

def BO_EI_RBF(train_X,train_Y, low_bound, up_bound, syn_function):
  tau_vec = params["tau"]
  tau = tau_vec[0]
  temp = train_X.numpy() 
  F_eval = obj_syn_fun(temp, syn_function)
  F_eval = np.array(F_eval)
  F_eval = torch.from_numpy(F_eval)
  F_eval = F_eval[0:2]
  iter = 200
  for j in range(iter):
    [Xstar, sc, obs] = BO_step_EI_RBF(train_X,train_Y,low_bound,up_bound, syn_function, tau)
    train_X = torch.cat((train_X,Xstar),0)
    obs = torch.reshape(obs ,(1,1))
    train_Y = torch.cat((train_Y,obs),0)
    F_eval = torch.cat((F_eval, sc))
  obj_f_list = F_eval.numpy()
  return obj_f_list

def BO_step_EI_Matern(train_X,train_Y, low_bound, up_bound, syn_function, tau, nu_val):
  
  train_Y2 = torch.squeeze(train_Y)

  # Botorch GP
  cov_module = gpytorch.kernels.MaternKernel(nu=nu_val)
  gp = SingleTaskGP(train_X, train_Y, covar_module = cov_module)
  mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
  fit_gpytorch_model(mll)
  

  # acquistion functions

  EI = ExpectedImprovement(gp, best_f= torch.max(train_Y))

  #Optimize acquisition functions
  
  bounds = torch.stack([torch.zeros(train_X.shape[1])+torch.tensor(low_bound), torch.zeros(train_X.shape[1])+torch.tensor(up_bound)])
  candidate_EI, acq_value_EI = optimize_acqf(
    EI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
  )
  candidate_EI_num = candidate_EI.numpy()
  candidate_EI_num = np.float64(candidate_EI_num)
  score_EI = obj_syn_fun(candidate_EI_num, syn_function)
  obs_EI = syn_fun(candidate_EI_num, syn_function, tau)
  score_EI = np.array(score_EI)
  obs_EI = np.array(obs_EI)
  score_EI = torch.from_numpy(score_EI)
  obs_EI = torch.from_numpy(obs_EI)


  return candidate_EI, score_EI, obs_EI

def BO_EI_Matern(train_X,train_Y, low_bound, up_bound, syn_function, nu_val):
  tau_vec = params["tau"]
  tau = tau_vec[0]
  temp = train_X.numpy() 
  F_eval = obj_syn_fun(temp, syn_function)
  F_eval = np.array(F_eval)
  F_eval = torch.from_numpy(F_eval)
  F_eval = F_eval[0:2]
  iter = 200
  for j in range(iter):
    [Xstar, sc, obs] = BO_step_EI_Matern(train_X,train_Y,low_bound,up_bound, syn_function, tau, nu_val)
    train_X = torch.cat((train_X,Xstar),0)
    obs = torch.reshape(obs ,(1,1))
    train_Y = torch.cat((train_Y,obs),0)
    F_eval = torch.cat((F_eval, sc))
  obj_f_list = F_eval.numpy()
  return obj_f_list

def BO_step_EI_ensemble(train_X,train_Y, low_bound, up_bound, syn_function, tau):
  train_Y2 = torch.squeeze(train_Y)
  M = 4
  #1st EI learner 
  gp1 = SingleTaskGP(train_X, train_Y)
  mll1 = ExactMarginalLogLikelihood(gp1.likelihood, gp1)
  fit_gpytorch_model(mll1)
  mloglike1 = mll1(gp1(train_X), train_Y2).item()

  #2nd EI learner 
  cov_module2 = gpytorch.kernels.RBFKernel()
  gp2 = SingleTaskGP(train_X, train_Y, covar_module = cov_module2)
  mll2 = ExactMarginalLogLikelihood(gp2.likelihood, gp2)
  fit_gpytorch_model(mll2)
  mloglike2 = mll2(gp2(train_X), train_Y2).item()

  #3rd EI learner 
  cov_module3 = gpytorch.kernels.MaternKernel(nu=2.5)
  gp3 = SingleTaskGP(train_X, train_Y, covar_module = cov_module3)
  mll3 = ExactMarginalLogLikelihood(gp3.likelihood, gp3)
  fit_gpytorch_model(mll3)
  mloglike3 = mll3(gp3(train_X), train_Y2).item()

  #4rd EI learner 
  cov_module4 = gpytorch.kernels.MaternKernel(nu=1.5)
  gp4 = SingleTaskGP(train_X, train_Y, covar_module = cov_module4)
  mll4 = ExactMarginalLogLikelihood(gp4.likelihood, gp4)
  fit_gpytorch_model(mll4)
  mloglike4 = mll4(gp4(train_X), train_Y2).item()

  w_vec = np.array([mloglike1,mloglike2,mloglike3,mloglike4])
  w_vec = update_weights_EI(w_vec)
  w_pmf = np.squeeze(w_vec)
  #print(w_pmf)
  ind_exp = np.random.choice(np.arange(1, M+1), p=w_pmf)
  
  if ind_exp == 1:
    gp = gp1
  elif ind_exp == 2:
    gp = gp2
  elif ind_exp == 3:
    gp = gp3
  else:
    gp = gp4  

  EI = ExpectedImprovement(gp, best_f= torch.max(train_Y))

  #Optimize acquisition functions
  
  bounds = torch.stack([torch.zeros(train_X.shape[1])+torch.tensor(low_bound), torch.zeros(train_X.shape[1])+torch.tensor(up_bound)])
  candidate_EI, acq_value_EI = optimize_acqf(
    EI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
  )
  candidate_EI_num = candidate_EI.numpy()
  candidate_EI_num = np.float64(candidate_EI_num)
  score_EI = obj_syn_fun(candidate_EI_num, syn_function)
  obs_EI = syn_fun(candidate_EI_num, syn_function, tau)
  score_EI = np.array(score_EI)
  obs_EI = np.array(obs_EI)
  score_EI = torch.from_numpy(score_EI)
  obs_EI = torch.from_numpy(obs_EI)


  return candidate_EI, score_EI, obs_EI


def BO_EI_ensemble(train_X,train_Y, low_bound, up_bound, syn_function):
  tau_vec = params["tau"]
  tau = tau_vec[0]
  M = 4
  temp = train_X.numpy() 
  F_eval = obj_syn_fun(temp, syn_function)
  F_eval = np.array(F_eval)
  F_eval = torch.from_numpy(F_eval)
  F_eval = F_eval[0:2]
  iter = 200  
  for j in range(iter):
    [Xstar, sc, obs] = BO_step_EI_ensemble(train_X,train_Y,low_bound,up_bound, syn_function, tau)
    train_X = torch.cat((train_X,Xstar),0)
    obs = torch.reshape(obs ,(1,1))
    train_Y = torch.cat((train_Y,obs),0)
    F_eval = torch.cat((F_eval, sc))
  obj_f_list = F_eval.numpy()
  return obj_f_list  






Run experiments for a single run

In [ ]:
aver_regret_gpfull = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_gpfull,var_pred = main_full_Bayesian_GP(X,y,params,X0_points,bnds,syn_function)
  obj_f_list_gpfull = np.squeeze(obj_f_list_gpfull)
  regret_list_gpfull = simple_regret(obj_f_list_gpfull,benchmark_value)
  aver_regret_gpfull = aver_regret_gpfull + regret_list_gpfull
aver_regret_gpfull = aver_regret_gpfull/n_runs
print("Final value of full Bayesian GP:", aver_regret_gpfull[199])
print(aver_regret_gpfull)

In [ ]:
aver_regret_gpARD = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_gpARD,var_pred = main_GP_ARD(X,y,params,X0_points,bnds,syn_function, reinit_key)
  obj_f_list_gpARD = np.squeeze(obj_f_list_gpARD)
  regret_list_gpARD = simple_regret(obj_f_list_gpARD,benchmark_value)
  aver_regret_gpARD = aver_regret_gpARD + regret_list_gpARD
aver_regret_gpARD = aver_regret_gpARD/n_runs
print("Final value of GP with ARD:", aver_regret_gpARD[199])
print(aver_regret_gpARD)

In [ ]:
aver_regret_gpmult = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_gpmult,var_pred = main_GP_matern(X,y,params,X0_points,bnds,syn_function, reinit_key)
  obj_f_list_gpmult = np.squeeze(obj_f_list_gpmult)
  regret_list_gpmult = simple_regret(obj_f_list_gpmult,benchmark_value)
  aver_regret_gpmult = aver_regret_gpmult + regret_list_gpmult
aver_regret_gpmult = aver_regret_gpmult/n_runs
print("Final value of GP with Mattern:", aver_regret_gpmult[199])
print(aver_regret_gpmult)

In [ ]:
aver_regret_gpmult = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_gpmult,var_pred = main_GP_matern2(X,y,params,X0_points,bnds,syn_function, reinit_key)
  obj_f_list_gpmult = np.squeeze(obj_f_list_gpmult)
  regret_list_gpmult = simple_regret(obj_f_list_gpmult,benchmark_value)
  aver_regret_gpmult = aver_regret_gpmult + regret_list_gpmult
aver_regret_gpmult = aver_regret_gpmult/n_runs
print("Final value of GP with Mattern 2:", aver_regret_gpmult[199])
print(aver_regret_gpmult)

In [ ]:
aver_regret_exp3 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_exp3,var_pred = main_EXP3BO(X,y,params,X0_points,bnds,syn_function,reinit_key,benchmark_value)
  obj_f_list_exp3 = np.squeeze(obj_f_list_exp3)
  regret_list_exp3 = simple_regret(obj_f_list_exp3,benchmark_value)
  aver_regret_exp3 = aver_regret_exp3 + regret_list_exp3
aver_regret_exp3 = aver_regret_exp3/n_runs
print("Final value of EXP3BO:", aver_regret_exp3[199])
print(aver_regret_exp3)

In [ ]:
aver_regret_exp3 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_exp3,var_pred = main_EXP3BO_ardrbfmatern(X,y,params,X0_points,bnds,syn_function,reinit_key,benchmark_value)
  obj_f_list_exp3 = np.squeeze(obj_f_list_exp3)
  regret_list_exp3 = simple_regret(obj_f_list_exp3,benchmark_value)
  aver_regret_exp3 = aver_regret_exp3 + regret_list_exp3
aver_regret_exp3 = aver_regret_exp3/n_runs
print("Final value of EXP3BO with ARD:", aver_regret_exp3[199])
print(aver_regret_exp3)

In [ ]:
x_t,y_pred,obj_f_list_mult2,var_pred = main_EGP_rbfmaternard(X,y,params,X0_points,bnds,syn_function, reinit_key)
obj_f_list_mult2 = np.squeeze(obj_f_list_mult2)
n = np.arange(1,len(obj_f_list_mult2)+1)
plt.plot(n,obj_f_list_mult2,n,benchmark_value*np.ones(len(obj_f_list_mult2)))
plt.title(syn_function + str(' - EGP with diff. kern. types'))
plt.ylabel('objective function value')
plt.xlabel('iteration')
plt.show()

In [ ]:
aver_regret_mult2 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list_mult2,var_pred = main_EGP_rbfmaternard(X,y,params,X0_points,bnds,syn_function,reinit_key)
  obj_f_list_mult2 = np.squeeze(obj_f_list_mult2)
  regret_list_mult2 = simple_regret(obj_f_list_mult2,benchmark_value)
  aver_regret_mult2 = aver_regret_mult2 + regret_list_mult2
aver_regret_mult2 = aver_regret_mult2/n_runs
print("Final value of EGP with different kernel types including ARD:", aver_regret_mult2[199])
print(aver_regret_mult2)

In [ ]:
aver_regret = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t,y_pred,obj_f_list,var_pred = main_GP(X,y,params,X0_points,bnds,syn_function,reinit_key)
  obj_f_list = np.squeeze(obj_f_list)
  regret_list = simple_regret(obj_f_list,benchmark_value)
  aver_regret = aver_regret + regret_list
aver_regret = aver_regret/n_runs
print("Final value of GP:", aver_regret[199])
print(aver_regret)

In [ ]:
aver_regret2 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t2,y_pred2,obj_f_list_egp,var_pred2 = main_EGP(X,y,params,X0_points,bnds,syn_function,reinit_key)
  obj_f_list_egp = np.squeeze(obj_f_list_egp)
  regret_list2 = simple_regret(obj_f_list_egp,benchmark_value)
  aver_regret2 = aver_regret2 + regret_list2
aver_regret2 = aver_regret2/n_runs
print("Final value of EGP:", aver_regret2[199])
print(aver_regret2)

In [ ]:
aver_regret3 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t3,y_pred3,obj_f_list_bandit,var_pred3 = main_BanditBO(X,y,params,X0_points,bnds,syn_function,reinit_key)
  obj_f_list_bandit = np.squeeze(obj_f_list_bandit)
  regret_list3 = simple_regret(obj_f_list_bandit,benchmark_value)
  aver_regret3 = aver_regret3 + regret_list3
aver_regret3 = aver_regret3/n_runs
print("Final value of BanditBO:", aver_regret3[199])
print(aver_regret3)

In [ ]:
aver_regret4 = np.zeros(200)
n_runs = 1
for i in range(n_runs):
  x_t3,y_pred3,obj_f_list_bandit,var_pred3 = main_BanditBO_ardrbfmatern(X,y,params,X0_points,bnds,syn_function,reinit_key)
  obj_f_list_bandit = np.squeeze(obj_f_list_bandit)
  regret_list4 = simple_regret(obj_f_list_bandit,benchmark_value)
  aver_regret4 = aver_regret4 + regret_list4
aver_regret4 = aver_regret4/n_runs
print("Final value of BanditBO with ARD:", aver_regret4[199])
print(aver_regret4)

In [ ]:
train_X = torch.from_numpy(X)
train_Y = torch.from_numpy(y)
train_Y = torch.reshape(train_Y,(10,1))

In [ ]:
aver_regret_EI = np.zeros(202)
n_runs = 1
for i in range(n_runs):
  obj_f_list_EI = BO_EI_ensemble(train_X,train_Y, low_bound, up_bound, syn_function)
  obj_f_list_EI = np.squeeze(obj_f_list_EI)
  regret_list_EI = simple_regret(obj_f_list_EI,benchmark_value)
  aver_regret_EI = aver_regret_EI + regret_list_EI
aver_regret_EI = aver_regret_EI/n_runs
print("Final value of GP with EI-ensemble:", aver_regret_EI[199])
print(aver_regret_EI)

In [ ]:
aver_regret_EI = np.zeros(202)
n_runs = 1
for i in range(n_runs):
  obj_f_list_EI = BO_EI(train_X,train_Y, low_bound, up_bound, syn_function)
  obj_f_list_EI = np.squeeze(obj_f_list_EI)
  regret_list_EI = simple_regret(obj_f_list_EI,benchmark_value)
  aver_regret_EI = aver_regret_EI + regret_list_EI
aver_regret_EI = aver_regret_EI/n_runs
print("Final value of GP with EI-ARD (default):", aver_regret_EI[199])
print(aver_regret_EI)

In [ ]:
aver_regret_EI = np.zeros(202)
n_runs = 1
for i in range(n_runs):
  obj_f_list_EI = BO_EI_RBF(train_X,train_Y, low_bound, up_bound, syn_function)
  obj_f_list_EI = np.squeeze(obj_f_list_EI)
  regret_list_EI = simple_regret(obj_f_list_EI,benchmark_value)
  aver_regret_EI = aver_regret_EI + regret_list_EI
aver_regret_EI = aver_regret_EI/n_runs
print("Final value of GP with EI-RBF:", aver_regret_EI[199])
print(aver_regret_EI)

In [ ]:
aver_regret_EI = np.zeros(202)
n_runs = 1
for i in range(n_runs):
  obj_f_list_EI = BO_EI_Matern(train_X,train_Y, low_bound, up_bound, syn_function, 2.5)
  obj_f_list_EI = np.squeeze(obj_f_list_EI)
  regret_list_EI = simple_regret(obj_f_list_EI,benchmark_value)
  aver_regret_EI = aver_regret_EI + regret_list_EI
aver_regret_EI = aver_regret_EI/n_runs
print("Final value of GP with EI-Matern 1:", aver_regret_EI[199])
print(aver_regret_EI)

In [ ]:
aver_regret_EI = np.zeros(202)
n_runs = 1
for i in range(n_runs):
  obj_f_list_EI = BO_EI_Matern(train_X,train_Y, low_bound, up_bound, syn_function, 1.5)
  obj_f_list_EI = np.squeeze(obj_f_list_EI)
  regret_list_EI = simple_regret(obj_f_list_EI,benchmark_value)
  aver_regret_EI = aver_regret_EI + regret_list_EI
aver_regret_EI = aver_regret_EI/n_runs
print("Final value of GP with EI-Matern 2:", aver_regret_EI[199])
print(aver_regret_EI)